## Notebook to compute the vocal/instrumental probabilities in the Saraga 1.5 dataset

The Essentia library currently contains 4 models to predict vocal/instrumental parts in music - https://essentia.upf.edu/models.html#voice-instrumental - some quick tests with the vocal track from saraga 1.5 show that the discogs and the MusicNN models doesn't work with the data, the other models predict the vocal parts pretty good.


This notebook computes the vocal/instrumental probabilities with the VGG model and the YAMNET and stores them as a csv file.

A folder for one track:

- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.ctonic.txt',

- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.multitrack-mridangam-left.mp3',
- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.multitrack-vocal.mp3',
- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.mp3.mp3',
- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.multitrack-mridangam-right.mp3',
- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.json',
- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.multitrack-violin.mp3'

will then be extended with a two files:

- '../data/saraga1.5_carnatic/Vidya Subramanian at Arkay by Vidya Subramanian/Shloka/Shloka.vocalprobs.csv'


In [1]:
from glob import glob
from essentia.standard import MonoLoader, TensorflowPredictVGGish, TensorflowPredict2D
import librosa
import pandas as pd
SR = 16000 #all models require that samplerate

2024-05-16 14:58:40.911521: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-16 14:58:40.911542: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2024-05-16 14:58:41.512201: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 14:58:41.514092: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-05-16 14:58:41.596747

### Load models

The code requires a folder *essentia_weights* where the model weights are stored (download link see above).

In [2]:
#VGG
embedding_model_vgg = TensorflowPredictVGGish(graphFilename="essentia_weights/audioset-vggish-3.pb", output="model/vggish/embeddings")
vocal_model_vgg = TensorflowPredict2D(graphFilename="essentia_weights/voice_instrumental-audioset-vggish-1.pb", output="model/Softmax")

#YAMNET
embedding_model_yamnet = TensorflowPredictVGGish(graphFilename="essentia_weights/audioset-yamnet-1.pb", input="melspectrogram", output="embeddings")
vocal_model_yamnet = TensorflowPredict2D(graphFilename="essentia_weights/voice_instrumental-audioset-yamnet-1.pb", output="model/Softmax")

2024-05-16 14:58:42.882550: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-16 14:58:42.883108: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-16 14:58:42.883155: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-16 14:58:42.883187: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Strea

In [3]:
def predict_vocal_yamnet(audio):
    embeddings = embedding_model_yamnet(audio)
    print('yamnet', audio.shape, embeddings.shape)
    predictions = vocal_model_yamnet(embeddings)
    return predictions

def predict_vocal_vgg(audio):
    embeddings = embedding_model_vgg(audio)
    print('vgg', audio.shape, embeddings.shape)
    predictions = vocal_model_vgg(embeddings)
    return predictions

In [4]:
path_list = glob('../data/saraga1.5_carnatic/*/*/*multitrack-vocal.mp3')
len(path_list)

132

In [8]:
for path in path_list:
    audio, sr = librosa.load(path, sr = SR)
    prediction_yamnet = predict_vocal_yamnet(audio)
    prediction_vgg = predict_vocal_vgg(audio)

    save_dict = {'vocal_prediction_yamnet': [x[1] for x in prediction_yamnet],
                 'silence_prediction_yamnet': [x[0] for x in prediction_yamnet],
                 'vocal_prediction_vgg': [x[1] for x in prediction_vgg],
                 'silence_prediction_vgg': [x[0] for x in prediction_vgg]}
    df = pd.DataFrame(save_dict)
    save_path = path.split('multitrack-vocal')[0] + 'vocalprobs.csv'
    #df.to_csv(save_path)
    print(save_path)


2024-05-15 18:02:17.953507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-15 18:02:17.953659: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-15 18:02:17.953669: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-15 18:02:17.953678: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Strea

yamnet (35773550,) (2404, 1024)
vgg (35773550,) (2404, 128)
../data/saraga1.5_carnatic/Kuldeep Pai at Arkay by Kuldeep Pai/Thappi Bratikipova/Thappi Bratikipova.vocalprobs.csv


StreamExecutor with strength 1 edge matrix:
2024-05-15 18:02:17.997306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2024-05-15 18:02:17.997310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2024-05-15 18:02:33.646365: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-15 18:02:33.646511: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-15 18:02:33.646521: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

yamnet (3981875,) (267, 1024)
vgg (3981875,) (267, 128)
../data/saraga1.5_carnatic/Kuldeep Pai at Arkay by Kuldeep Pai/Gange Maam Pahi/Gange Maam Pahi.vocalprobs.csv


2024-05-15 18:02:34.727876: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-15 18:02:34.727886: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-15 18:02:34.727894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-05-15 18:02:34.727898: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2024-05-15 18:02:34.727902: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0: 

...

2024-05-15 18:05:23.135348: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


yamnet (12791387,) (859, 1024)
vgg (12791387,) (859, 128)
../data/saraga1.5_carnatic/Dr S Sundar at Arkay by S Sundar/Paahi Maam Sri/Paahi Maam Sri.vocalprobs.csv


2024-05-15 18:05:23.135496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-15 18:05:23.135507: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-15 18:05:23.135515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2024-05-15 18:05:23.135519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2024-05-15 18:05:23.135524: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0: 

yamnet (25837759,) (1736, 1024)


KeyboardInterrupt: 

In [18]:
3981875 / 267

14913.389513108614

In [9]:
import numpy as np
audio = np.random.rand(64000)
embeddings = embedding_model_yamnet(audio)
print('yamnet', audio.shape, embeddings.shape)
predictions = vocal_model_yamnet(embeddings)

yamnet (64000,) (4, 1024)


[   INFO   ] Warning: essentia can currently only accept numpy arrays of dtype "single". "signal" dtype is double. Precision will be automatically truncated into "single".
2024-05-16 14:59:08.175658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-16 14:59:08.175789: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-16 14:59:08.175799: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required lib

In [13]:
import warnings; warnings.simplefilter('ignore')

embedding_model_vgg = TensorflowPredictVGGish(graphFilename="essentia_weights/audioset-vggish-3.pb", output="model/vggish/embeddings")
audio = np.random.rand(16000)
embeddings = embedding_model_vgg(audio)
print(audio.shape, embeddings.shape)

audio = np.random.rand(160000)
embeddings = embedding_model_vgg(audio)
print(audio.shape, embeddings.shape)

(64000,) (4, 128)
(640000,) (42, 128)

2024-05-16 15:09:22.640130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-16 15:09:22.640267: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-16 15:09:22.640277: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-05-16 15:09:22.640286: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Strea

(16000,) (1, 128)
(160000,) (10, 128)


[   INFO   ] Warning: essentia can currently only accept numpy arrays of dtype "single". "signal" dtype is double. Precision will be automatically truncated into "single".
2024-05-16 15:09:24.779054: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-05-16 15:09:24.779297: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro T2000 computeCapability: 7.5
coreClock: 1.5GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 104.34GiB/s
2024-05-16 15:09:24.779306: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1766] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required lib

TypeError: 'tuple' object is not callable